### Crate a nested .js from .csv to import into MongoDB
#### CICO data: https://www.cliclavoro.gov.it/Barometro-Del-Lavoro/Pagine/Microdati-per-la-ricerca.aspx

In [1]:
# Required packages
import pandas as pd
import numpy as np
import json
import os
import pymongo

In [2]:
# Import data (.csv format)
df = pd.read_csv('C:/Users/Nicola Caravaggio/OneDrive/Desktop/Ciu_Tos_Roma3/CICO/data_cico.csv')
print ('Data correctly imported')
df.head()

# to import all as string, add ", dtype = str" after the path

Data correctly imported


,cfdatore_crip,cflavoratore_crip,rapporto_datainizio,annoattivazione,annonascita,codgenere,regione_nascita,codcittadinanza,codtitolostudio,codregionedomicilio,...,codccnl,codagevolazione,idsociolavoratore,dtcessazioneeffettiva,codmotivocessazioneco,dtfineprevista,dttrasformazione,codtipotrasformazione,numeroproroghe,coef_uni
0,48D49BDAC8D8F2B2D63A6122F2B4D332,4C30B254455CDB5FF6E53857746D633E,06jul2017,2017,1953,Femmina,Campania,Italia,20,Campania,...,235,999,No,10jul2017,254.0,10jul2017,NaN,NaN,0,7.604166
1,90FF9B696EC2BD0419792F33FA3EBCEE,4CC0E3D4A897A6661D35D4D30D07F57A,01dec2011,2011,1988,Maschio,Sicilia,Italia,70,Sicilia,...,999,999,No,31mar2012,254.0,31mar2012,NaN,NaN,0,7.625000
2,D23485B84D16E0F9CCDB7B7F013B4F76,57E6FA30170F062738665433915DEAFD,22apr2015,2015,1960,Femmina,Campania,Italia,10,Campania,...,11,999,No,30jun2015,254.0,30jun2015,NaN,NaN,0,7.604166
3,182D316EB9A78C40E0CF5A2D5C25056C,49AF87408CC71FBB51C01D6D0C90BC3B,01aug2016,2016,1990,Maschio,Estero,Extra UE,40,Veneto,...,257,999,No,22jan2018,10.0,NaN,NaN,NaN,0,5.436979
4,90D5C65C1557742DBE2808A10956456D,64D130D252AE4739EC6D7EF9197D76DE,01jun2011,2011,1970,Maschio,Estero,Extra UE,0,Campania,...,272,999,No,31jan2012,12.0,NaN,NaN,NaN,0,5.451875


In [ ]:
# NaN to null
df = df.where((pd.notnull(df)), None)
# Alternatively, Nan to ""
#df = df.replace(np.nan, 'null', regex = True)

In [ ]:
# Check columns' types
df.info(verbose = True)

In [ ]:
# Convert int64 (not supported in MongoDB) into float64
df["annoattivazione"] = df["annoattivazione"].astype(np.float64)
df["annonascita"] = df["annonascita"].astype(np.float64)
df["codtitolostudio"] = df["codtitolostudio"].astype(np.float64)
df["codprovincialavoro"] = df["codprovincialavoro"].astype(np.float64)
df["codtipocontratto"] = df["codtipocontratto"].astype(np.float64)
df["codccnl"] = df["codccnl"].astype(np.float64)
df["codagevolazione"] = df["codagevolazione"].astype(np.float64)
df["numeroproroghe"] = df["numeroproroghe"].astype(np.float64)

In [ ]:
# Descriptive statistics
df.describe()

#### Generate a .json-type file from the imported .csv with nested arrays

In [ ]:
# Encode variables for .json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

# Create a document for each worker (cflavoratore_crip) with a nested array for each working contract (attivazioni)
finalList = []
finalDict = {}
grouped = df.groupby(['cflavoratore_crip', 
                      'annonascita',
                      'codgenere',
                      'regione_nascita',
                      'codcittadinanza',
                      'codtitolostudio',
                      'coef_uni'
                     ])
for key, value in grouped:

    dictionary = {}

    j = grouped.get_group(key).reset_index(drop=True)
    dictionary['cflavoratore_crip'] = j.at[0, 'cflavoratore_crip']
    dictionary['annonascita'] = j.at[0, 'annonascita']
    dictionary['codgenere'] = j.at[0, 'codgenere']
    dictionary['regione_nascita'] = j.at[0, 'regione_nascita']
    dictionary['codcittadinanza'] = j.at[0, 'codcittadinanza']
    dictionary['codtitolostudio'] = j.at[0, 'codtitolostudio']
    dictionary['coef_uni'] = j.at[0, 'coef_uni']
    
    dictList = []
    anotherDict = {}
    for i in j.index:

        anotherDict['cfdatore_crip'] = j.at[i, 'cfdatore_crip']
        anotherDict['rapporto_datainizio'] = j.at[i, 'rapporto_datainizio']
        anotherDict['codregionedomicilio'] = j.at[i, 'codregionedomicilio']
        anotherDict['codregionelavoro'] = j.at[i, 'codregionelavoro']
        anotherDict['codprovincialavoro'] = j.at[i, 'codprovincialavoro']
        anotherDict['codsettore'] = j.at[i, 'codsettore']
        anotherDict['codtipocontratto'] = j.at[i, 'codtipocontratto']
        anotherDict['codtipoorario'] = j.at[i, 'codregionedomicilio']
        anotherDict['codqualificaprofessionale'] = j.at[i, 'codqualificaprofessionale']
        anotherDict['codccnl'] = j.at[i, 'codccnl']
        anotherDict['codagevolazione'] = j.at[i, 'codagevolazione']
        anotherDict['idsociolavoratore'] = j.at[i, 'idsociolavoratore']
        anotherDict['dtcessazioneeffettiva'] = j.at[i, 'dtcessazioneeffettiva']
        anotherDict['dtfineprevista'] = j.at[i, 'dtfineprevista']
        anotherDict['dttrasformazione'] = j.at[i, 'dttrasformazione']
        anotherDict['codtipotrasformazione'] = j.at[i, 'codtipotrasformazione']
        anotherDict['numeroproroghe'] = j.at[i, 'numeroproroghe']        

        dictList.append(anotherDict.copy())

    dictionary['attivazioni'] = dictList

    finalList.append(dictionary)

# Note: the following commands are useful just to show the structure, 
# the objsect that will be imported into MongoDB is 'finalList'
cico_final = (json.dumps(finalList, indent = 4, cls = NpEncoder))
print(json.dumps(finalList, indent = 4, cls = NpEncoder))

#### Import data into MongoDB

In [ ]:
# Import data into MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["cico"]

print(db.list_collection_names())
print("The collection does exist?")
print('datacico' in db.list_collection_names())    
collection = db['datacico']
collection.estimated_document_count() == 0
collection.drop()

col = db['datacico']

result = col.insert_many(finalList)

# Number of object inserted
inserted = result.inserted_ids
print("Number of inserted objects:") 
len(inserted)

In [ ]:
# To save as .js format
#with open('C:/Users/Nicola Caravaggio/OneDrive/Desktop/Ciu_Tos_Roma3/CICO/data_cico.js', 'w') as 
#f: json.dump(cico_final, f, indent = 4, cls = NpEncoder)